# Lesson 2: Implementing Atomic Transactions with Watch

# Introduction to `watch` in Redis

Welcome back! You’ve now learned how to build and execute basic transactions in Redis. This next lesson takes you a step further by introducing the `watch` command. This command will help you implement more controlled and conditional transactions. It is essential for scenarios where you need to monitor certain keys and ensure that operations are completed only when specific conditions are met.

## What You'll Learn
In this unit, you will delve into the functionality of the `watch` command in Redis. Here’s a quick overview of what you will learn:

- **Setting Up `watch`**: Understand the importance of monitoring keys to control transaction execution.
- **Implementing Conditional Updates**: Write functions that use `watch` to implement safer and more conditional updates to your Redis data.

Let's take a look at a practical example of how to use `watch` in your code.

```python
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

def update_balance(user_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                pipe.watch(f'balance:{user_id}')
                balance = int(pipe.get(f'balance:{user_id}') or 0)
                pipe.multi()
                pipe.set(f'balance:{user_id}', balance + increment)
                pipe.execute()
                break
            except redis.WatchError as e:
                print(f"Retrying transaction: {e}")
                continue

client.set('balance:1', 100)
update_balance(1, 50)

value = client.get('balance:1').decode('utf-8')
print(f"Updated balance for user 1: {value}")
```

In this example, we start by **watching** the `balance:{user_id}` key to monitor changes. If another client modifies the value before executing the transaction, the `pipe.execute()` will fail, and the transaction will retry. This ensures that your balance updates are consistent.

### Breakdown of Each Step:

1. **Define the Function**: The `update_balance` function takes `user_id` and `increment` as arguments.
2. **Create a Pipeline**: We use `client.pipeline()` to execute multiple commands in a single transaction.
3. **Monitor Key**: Inside the `while` loop, we use the `watch` command to monitor the `balance:{user_id}` key, ensuring that no other client modifies it during the transaction.
4. **Get the Current Balance**: We retrieve the current balance value using `pipe.get()` and set it to `0` if it doesn't exist.
5. **Start the Transaction**: The `pipe.multi()` command starts the transaction block, allowing multiple commands to be executed atomically.
6. **Update the Balance**: We update the balance by adding the `increment` value to the current balance.
7. **Execute the Transaction**: The `pipe.execute()` command executes the transaction.
8. **Handle Conflicts**: If another client changes the balance key before the transaction is executed, a `redis.WatchError` exception is raised, and the transaction is retried using the `continue` statement.

### How the Function is Used:

1. We set the initial balance for user 1 to `100`.
2. We call the `update_balance` function with `user_id=1` and `increment=50` to increase the balance by 50.
3. Finally, we retrieve the updated balance for user 1 and print it.

## Why It Matters

Mastering the `watch` command is crucial for several reasons:

- **Optimized Data Integrity**: Using `watch` ensures that actions only occur if specific conditions are met, allowing for safer updates.
- **Conditional Logic**: You can tailor your Redis transactions to proceed only when specific keys maintain expected values, adding sophistication and precision to your operations.
- **Error Handling**: `watch` helps in avoiding conflicts and managing errors when multiple clients try to update the same data.

Using `watch` effectively enables you to write more robust and reliable applications, safeguarding against potential race conditions and ensuring that concurrent updates do not interfere with each other.

Ready to get hands-on and explore further? Let’s move on to the practice section and apply these commands in various scenarios to solidify your understanding.

## Watch and Update User Balance

## Updating User Balance with Redis

## Abort Transaction on Negative Balance

## Update Likes with Redis Transactions

## Update Points with Redis Transactions